# 0319 captioning and visual QA

## Generate a Caption

In [ ]:
image_path = "/content/KakaoTalk_Photo_2024-03-19-16-08-35.jpeg"

from PIL import Image

image = Image.open(image_path)
image

In [3]:
from transformers import pipeline
# Salesforce/blip-image-captioning-large
import warnings
warnings.filterwarnings(action='ignore')

image_to_text = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

generated_text = image_to_text(image_path)[0]["generated_text"]
print(generated_text)

people sitting at a table with laptops in a large room


## Question and Answering about the image

In [4]:
question = 'What color is the women\'s top? Is same with the color of man\'s top?'
print("question:",question)

question: What color is the women's top? Is same with the color of man's top?


### IMAGE QA - Salesforce/blip2-opt-2.7b

In [ ]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch
from transformers import ViltProcessor
from transformers import ViltForQuestionAnswering

model_checkpoint = "Salesforce/blip2-opt-2.7b"

processor_auto = AutoProcessor.from_pretrained(model_checkpoint)

# prepare inputs
model_blip2 = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

device = "cuda" if torch.cuda.is_available() else "cpu"
model_blip2.to(device)

In [9]:
prompt = f"You are an AI assistant of blind. Please make some decription about the image based on the question. Question: {question} Answer:"

inputs_auto = processor_auto(images=image, text=prompt, return_tensors="pt").to(device, torch.float16)

generated_ids = model_blip2.generate(**inputs_auto, max_new_tokens=100, num_beams=10, #1보다 큰 값을 지정
    early_stopping=True #EOS토큰이 나오면 생성을 중단
                               )
generated_text = processor_auto.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

The woman's top is blue and the man's top is black


### IMAGE QA - dandelin/vilt-b32-finetuned-vqa

In [27]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

# prepare image + question
image = image
text = question

processor_vilt = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model_vilt = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# prepare inputs
encoding_vilt = processor_vilt(image, text, return_tensors="pt")

# forward pass
outputs_vilt = model_vilt(**encoding)
logits = outputs_vilt.logits
idx = logits.argmax(-1).item()
print("Predicted answer:", model_vilt.config.id2label[idx])

Predicted answer: tan
